In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math

# Unfortunately, knn functions prompt "future warnings", so the commands below turn these off
import warnings
warnings.filterwarnings('ignore') 

In [4]:
## Note that textfile containing these data uses a tab delimiter to separate the label and message
sms = pd.read_csv("sms_spam.txt", sep='\t', names=['Label','Message'])

## Train-test split
from sklearn.model_selection import train_test_split
train, test = train_test_split(sms, test_size=0.2, random_state=8)

## Split outcome from predictors
train_y = (train['Label'] == 'spam').astype(int)
train_msg = train['Message']

## Feature engineering functions
def get_num(text):
    return sum(map(str.isdigit, text))/len(text)
def cap_percent(text):
    return sum(map(str.isupper, text))/len(text)
def alpha_percent(text):
    return sum(map(str.isalnum, text))/len(text)

## Define "first_word" function
def first_word(text):
    return text.split(sep=' ')[0].lower().replace('!','')

## Create data frame with these features
d = {'prop_num': train_msg.apply(get_num),
     'prop_cap': train_msg.apply(cap_percent),
     'prop_alp': train_msg.apply(alpha_percent),
    'first': train_msg.apply(first_word)}
train_X = pd.DataFrame(d)

## Assemble final training X data
train_X_ohe = pd.get_dummies(train_X, columns=['first'])
train_X = train_X_ohe[['prop_num','prop_cap', 'prop_alp', 'first_urgent','first_free']]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler

## Defining the models
model1 = LogisticRegression(penalty='none')
model2 = DecisionTreeClassifier(max_depth=5)
model3 = Pipeline([('scaler', StandardScaler()),
                  ('model', KNeighborsClassifier())])
                  
## Creating the ensemble
my_ensemble = VotingClassifier(estimators=[('logr', model1),
                                           ('tree', model2), 
                                           ('knn', model3)],
                               voting='soft')

In [6]:
from sklearn.model_selection import cross_val_score
print(np.average(cross_val_score(my_ensemble, train_X, train_y, scoring='f1', cv=5)))

## Individual models
print([np.average(cross_val_score(model1, train_X, train_y, scoring='f1', cv=5)),
       np.average(cross_val_score(model2, train_X, train_y, scoring='f1', cv=5)),
       np.average(cross_val_score(model3, train_X, train_y, scoring='f1', cv=5))])

0.9128026222243044
[0.8453849485158784, 0.906203023823853, 0.9084395064728428]


In [7]:
#1-a


params = {'logr__penalty': ['none','l2'], 
          'tree__max_depth': [4,5,6,7],
         'knn__model__n_neighbors': [3,6,10,15],
         'knn__model__weights': ['distance','uniform'],
         'voting': ['soft','hard']}

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(my_ensemble, param_grid=params, cv=5, scoring = 'f1').fit(train_X, train_y)
print(grid.best_estimator_)
print(grid.best_score_)

VotingClassifier(estimators=[('logr', LogisticRegression(penalty='none')),
                             ('tree', DecisionTreeClassifier(max_depth=5)),
                             ('knn',
                              Pipeline(steps=[('scaler', StandardScaler()),
                                              ('model',
                                               KNeighborsClassifier(n_neighbors=3,
                                                                    weights='distance'))]))],
                 voting='soft')
0.9167655692653746


In [8]:
#1-b
test_y = (test['Label'] == 'spam').astype(int)
test_msg = test['Message']

## Create data frame with these features
d = {'prop_num': test_msg.apply(get_num),
     'prop_cap': test_msg.apply(cap_percent),
     'prop_alp': test_msg.apply(alpha_percent),
    'first': test_msg.apply(first_word)}

test_X = pd.DataFrame(d)

## Assemble final training X data
test_X_ohe = pd.get_dummies(test_X, columns=['first'])
test_X = test_X_ohe[['prop_num','prop_cap', 'prop_alp', 'first_urgent','first_free']]

y_pred = grid.best_estimator_.predict(test_X)

from sklearn.metrics import f1_score
f1_score(test_y, y_pred)


0.9328859060402686

In [9]:
# 2a

## Create list of models in the ensemble

from sklearn.pipeline import Pipeline 
## Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler

pipe_knn = Pipeline([
('scaler', MinMaxScaler()),
('model', LogisticRegression(penalty='none', solver='newton-cg')),
])


from sklearn.preprocessing import PolynomialFeatures
pipe_log = Pipeline([('expander', PolynomialFeatures(degree=2)),
                  ('model', LogisticRegression(penalty='none'))])
                  
                  

base_models = [('logreg', pipe_log),
              ('tree', DecisionTreeClassifier(max_depth=5)),
              ('knn', pipe_knn)]

## Final model used to aggregate predicted probs
my_final_estimator = DecisionTreeClassifier(max_depth=3)

## Create stack
from sklearn.ensemble import StackingClassifier
my_stack = StackingClassifier(estimators = base_models, final_estimator = my_final_estimator, 
                              stack_method ='predict_proba', cv=5)

In [12]:
# 2b
## Fit and Evaluate (cv is done internally in StackingClassifier)
fitted_stack = my_stack.fit(train_X, train_y)
cv_stacked_preds = fitted_stack.predict(train_X)

from sklearn.metrics import f1_score
print(f1_score(cv_stacked_preds, train_y))

0.9209621993127148


In [13]:
# 2-c 
my_stack2 = StackingClassifier(estimators = base_models, final_estimator = LogisticRegression(), 
                              stack_method ='predict_proba', cv=5)

fitted_stack = my_stack2.fit(train_X, train_y)

cv_stacked_preds = fitted_stack.predict(train_X)

print(f1_score(cv_stacked_preds, train_y))

0.9153439153439153


the change did not improve the cross-validated f-1 score. 

In [39]:
#2-d
coef = my_stack2.final_estimator_.coef_
intercept = my_stack2.final_estimator_.intercept_
print(f'coefficient is {coef}, intercept is {intercept}')

coefficient is [[3.95164476 4.03163514 0.52616069]], intercept is [-4.23101673]


In [41]:
my_stack2

StackingClassifier(cv=5,
                   estimators=[('logreg',
                                Pipeline(steps=[('expander',
                                                 PolynomialFeatures()),
                                                ('model',
                                                 LogisticRegression(penalty='none'))])),
                               ('tree', DecisionTreeClassifier(max_depth=5)),
                               ('knn',
                                Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('model',
                                                 LogisticRegression(penalty='none',
                                                                    solver='newton-cg'))]))],
                   final_estimator=LogisticRegression(),
                   stack_method='predict_proba')

predicted class probabilities from the logistic regression model are multiplied by 3.95164476, so they contribute second highest to the ensemble's scores; predicted class probabilities from the decision tree model are multiplied by 4.03, so they contribute most to the ensemble's scores; finally, the predicted class probabilities from the  k-nearest neighbors model are multiplied by 0.52616069, so they contribute the least to the ensemble's scores. 

// ask about interpretation about intercepts. 

In [34]:
# 3-a

## Read IC home sales data
ic = pd.read_csv("IowaCityHomeSales.csv")

## Split to training and testing sets
from sklearn.model_selection import train_test_split
train_ic, test_ic = train_test_split(ic, test_size=0.2, random_state=7)

## Create outcome var
train_y_ic = train_ic['sale.amount']

## Create predictor matrix (numeric predictors only for simplicity, but we could use OHE if we wanted to)
train_X_ic= train_ic.select_dtypes("number").drop('sale.amount',axis=1)

## Create list of models in the ensemble
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
base_models = [('linreg', LinearRegression()),
              ('tree', DecisionTreeRegressor(max_depth=5)),
              ('knn', KNeighborsRegressor(n_neighbors=20))]

## Final model to aggregate base models in the ensemble
my_final_estimator = DecisionTreeRegressor(max_depth=3)

## Stacked regressor
from sklearn.ensemble import StackingRegressor
reg_stack =  StackingRegressor(base_models, final_estimator = my_final_estimator, cv=5)

## Fit and Evaluate (cv is done internally)
fitted_stack = reg_stack.fit(train_X_ic, train_y_ic)
cv_stacked_preds = fitted_stack.predict(train_X_ic)


In [42]:
reg_stack.final_estimator_

DecisionTreeRegressor(max_depth=3)

#3-b


When an observation is given, all the predictors are fed into three models. All the models will make predictions. To get a final result, final estimator, decision tree regressor, will aggregate the results of three models. 

In [45]:
reg_stack

StackingRegressor(cv=5,
                  estimators=[('linreg', LinearRegression()),
                              ('tree', DecisionTreeRegressor(max_depth=5)),
                              ('knn', KNeighborsRegressor(n_neighbors=20))],
                  final_estimator=DecisionTreeRegressor(max_depth=3))

In [49]:
#4-a
from sklearn.ensemble import RandomForestClassifier
my_forest = RandomForestClassifier(max_depth = 3, min_samples_split = 10, 
                                   max_features = 2, n_estimators = 200, 
                                   random_state = 0, oob_score = True)
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline 

pipe = Pipeline([
    ('classifier', my_forest)
])

parms = {
         'classifier__max_depth': [2, 3, 4],
         'classifier__min_samples_split': [5, 10, 15],
         'classifier__max_features': [1, 2]
        }

grid = GridSearchCV(
    pipe, parms, cv = 3, scoring = 'accuracy').fit(train_X, train_y)
print(grid.best_estimator_)
grid.best_score_


Pipeline(steps=[('classifier',
                 RandomForestClassifier(max_depth=4, max_features=2,
                                        min_samples_split=5, n_estimators=200,
                                        oob_score=True, random_state=0))])


0.9717300113441881

In [56]:
#4-b
base_models2 = [('linreg', LinearRegression()),
                ('tree', KNeighborsRegressor(n_neighbors = 5, weights = 'uniform')),
                ('knn', KNeighborsRegressor(n_neighbors = 20))]

## Final model to aggregate base models in the ensemble
my_final_estimator = DecisionTreeClassifier(max_depth = 3)

## Stacked regressor
from sklearn.ensemble import StackingRegressor
reg_stack =  StackingRegressor(base_models2, final_estimator = my_final_estimator, cv = 3)

In [57]:
reg_stack

StackingRegressor(cv=3,
                  estimators=[('linreg', LinearRegression()),
                              ('tree', KNeighborsRegressor()),
                              ('knn', KNeighborsRegressor(n_neighbors=20))],
                  final_estimator=DecisionTreeClassifier(max_depth=3))

In [58]:
# 4-c
foura = grid.best_estimator_
fourb = reg_stack


pipe = Pipeline([
    ('model', None)
])

parms = {
         'model': [foura, fourb],
        }

grid = GridSearchCV(
    pipe, parms, cv = 3, scoring = 'f1').fit(train_X, train_y)
print(grid.best_estimator_)



Pipeline(steps=[('model',
                 Pipeline(steps=[('classifier',
                                  RandomForestClassifier(max_depth=4,
                                                         max_features=2,
                                                         min_samples_split=5,
                                                         n_estimators=200,
                                                         oob_score=True,
                                                         random_state=0))]))])


0.879428525313111

In [60]:
# 4-d

from sklearn.metrics import accuracy_score

test_target_predicted = grid.best_estimator_.fit(train_X, train_y).predict(test_X)

accuracy = accuracy_score(test_y, test_target_predicted)

y_pred = grid.best_estimator_.predict(test_X)

from sklearn.metrics import f1_score
f1 = f1_score(test_y, y_pred)
print(f'accuracy is {accuracy}, f1 is {f1}')


accuracy is 0.97847533632287, f1 is 0.9172413793103448
